In [1]:
from dotenv import load_dotenv
import asyncio

load_dotenv()

True

In [ ]:
import os

In [ ]:
import requests
import json


def stream_with_reasoning(question, api_token):
    url = "https://openrouter.ai/api/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {api_token}",
        "Content-Type": "application/json",
    }

    payload = {
        "model": "deepseek/deepseek-r1",
        "messages": [{"role": "user", "content": question}],
        "stream": True,
        "include_reasoning": True,
    }

    buffer = ""
    is_reasoning = True  # Track whether we're in reasoning or content phase
    print("Reasoning:", end="\n\n", flush=True)

    with requests.post(url, headers=headers, json=payload, stream=True) as r:
        for chunk in r.iter_content(chunk_size=1024, decode_unicode=True):
            buffer += chunk
            while True:
                try:
                    line_end = buffer.find("\n")
                    if line_end == -1:
                        break

                    line = buffer[:line_end].strip()
                    buffer = buffer[line_end + 1 :]

                    if line.startswith("data: "):
                        data = line[6:]
                        if data == "[DONE]":
                            break

                        try:
                            data_obj = json.loads(data)
                            delta = data_obj["choices"][0].get("delta", {})

                            # Check for phase transition
                            if (
                                is_reasoning
                                and "content" in delta
                                and delta.get("content")
                            ):
                                is_reasoning = False
                                print("\n\nResponse:", end="\n\n", flush=True)

                            # Handle reasoning phase
                            if is_reasoning and "reasoning" in delta:
                                reasoning = delta["reasoning"]
                                if reasoning and reasoning.strip():
                                    print(reasoning, end="", flush=True)

                            # Handle content phase
                            elif not is_reasoning and "content" in delta:
                                content = delta["content"]
                                if content and content.strip():
                                    print(content, end="", flush=True)

                        except json.JSONDecodeError:
                            pass
                except Exception as e:
                    print(f"\nError: {str(e)}")
                    break


question = "How would you build the tallest building ever?"
api_token = os.getenv("OPENROUTER_API_KEY")
stream_with_reasoning(question, api_token)

Reasoning:

Okay, so I need to figure out how to build the tallest building ever. Hmm, where to start. Well, first, I guess I need to think about what makes a building tall and stable. Maybe look at existing skyscrapers like the Burj Khalifa or the Jeddah Tower. What do they do that allows them to reach such heights?

Alright, so the key factors must be materials, design, foundation, wind resistance, elevators, and safety systems. Let's break this down step by step.

Materials: Skyscrapers use strong materials. Steel and concrete are common. But maybe newer materials like carbon fiber or composites could be stronger and lighter. Wait, but are those materials feasible for such large-scale construction? Cost might be an issue. Maybe a combination? High-strength concrete for the core and steel for the frame?

Design: The shape is important. The Burj Khalifa has a Y-shaped plan to reduce wind loads. Aerodynamic designs help with wind resistance. Maybe a tapered structure, getting narrower 